In [16]:
from pyspark.sql.functions import col, avg
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer

In [17]:
reviews_df = spark.read.json('../data/reviews')

In [18]:
train_df, test_df = reviews_df.randomSplit(
    weights=[0.75, 0.25]
)

In [19]:
print(train_df.count())
print(test_df.count())

1931928
645370


In [20]:
from pyspark.ml.recommendation import ALS

In [21]:
train_df.agg(avg(col('stars'))).first()[0]

3.700451569623713

In [22]:
user_groups = train_df.groupBy('user_id')

In [23]:
[(row['user_id'], row['avg(stars)']) for row in user_groups.avg('stars').collect()]

[('2XDTpvVKW2aJwlyyNWwGsg', 3.6666666666666665),
 ('EaBKe-8LB-NHuH7Us-QhGw', 3.8378378378378377),
 ('fDAZoT0PRyd0K9TKerBxsQ', 2.0),
 ('zWWcik1fRPZviBCQLC26FQ', 3.211340206185567),
 ('eeVuxeWBuokW26A4S68C3Q', 2.6666666666666665),
 ('Z1ujtbw8Edcrgcr6ebSyOA', 2.6666666666666665),
 ('BGzavA_ddMr-jGmhArv7fg', 3.6008771929824563),
 ('60_lsQJ8JEGfZeXcTgKn8Q', 4.0476190476190474),
 ('NkENycDoq8acyrJl4jNwfA', 4.666666666666667),
 ('ZjaNE5PtYk8j2K1kOpr2Ig', 3.2580645161290325),
 ('iZwoTW2DoMRNtpAYqChIFg', 3.1666666666666665),
 ('HrZbzPHFCoeS1gbsnkAqHQ', 4.0),
 ('D2ljL5ejuqpa4f8fnZ1qjw', 3.7739726027397262),
 ('-mTmwfY9K9zcPnVmTltH3w', 4.5),
 ('zHJ2b0q5eucGugUWPnQYoQ', 5.0),
 ('iy8snpt14cWUXvcX-5xNXg', 4.0),
 ('4tF9R_a5je3fPwan8PIaPQ', 4.5),
 ('WdP5vTFdRmHKEkbGHFoO0g', 1.0),
 ('t89f0VSfCqYF8QM8UfFHEQ', 4.0),
 ('fIwRq2ebG_Ve1NTwfMVU7g', 3.0),
 ('W584cDDHqw_PYd5_l3F-nA', 5.0),
 ('KuGQIxKfGiHLNRuWLuPpXA', 3.806451612903226),
 ('-hnBzgVoRoqLrGVSxO0ilg', 3.260869565217391),
 ('A8WnomchAmsRTXnGo47lpw',

In [24]:
train_df.describe().show()

+-------+--------------------+------------------+--------------------+
|summary|         business_id|             stars|             user_id|
+-------+--------------------+------------------+--------------------+
|  count|             1931928|           1931928|             1931928|
|   mean|                null| 3.700451569623713|                null|
| stddev|                null|1.3296492687453587|                null|
|    min|--6MefnULPED_I942...|                 1|---1lKK3aKOuomHnw...|
|    max|zzzaIBwimxVej4tY6...|                 5|zzzmshdEWLFCApxET...|
+-------+--------------------+------------------+--------------------+



In [28]:
als_model = ALS(
    itemCol='business_idx',
    userCol='user_idx',
    ratingCol='stars',
    nonnegative=True,
    regParam=0.1,
    rank=10
)

In [29]:
user_idx_mdl = (
    StringIndexer(inputCol="user_id", outputCol="user_idx")
    .fit(reviews_df)
)
business_idx_mdl = (
    StringIndexer(inputCol="business_id", outputCol="business_idx")
    .fit(reviews_df)
)

train_df1 = business_idx_mdl.transform(user_idx_mdl.transform(train_df))
test_df1 = business_idx_mdl.transform(user_idx_mdl.transform(test_df))

In [30]:
model = als_model.fit(train_df1)

Py4JJavaError: An error occurred while calling o276.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 34.0 failed 1 times, most recent failure: Lost task 2.0 in stage 34.0 (TID 550, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 175415 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
	at org.apache.spark.scheduler.

In [13]:
sc

In [14]:
spark